# Prompt Versions Viewer

This notebook provides a user-friendly interface to explore and inspect different LLM prompt versions used in the ALeRCE Text-to-SQL pipeline.

In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML, Markdown
import pandas as pd
import difflib
import html
import re
import json
from prompts.prompts_pipeline import prompt_versions, get_prompt_version
from utils.utils import get_db_schema_prompt
from utils.pipeline_utils import load_prompt
from constants import DifficultyLevel, SQLErrorType
from utils.html_diff import get_html_diff

## Helper Functions for Rendering Prompts

In [2]:
# Group versions by their prefix
def group_versions_by_prefix():
    """Group prompt versions by their prefix (e.g., sl_, dir_, etc.)"""
    grouped = {}
    for version in prompt_versions.keys():
        prefix = version.split('_')[0]
        if prefix not in grouped:
            grouped[prefix] = []
        grouped[prefix].append(version)
    
    # Sort the versions within each prefix
    for prefix in grouped:
        grouped[prefix].sort()
    
    return grouped

# Extract display name for each version
def get_version_display_name(version_id):
    """Get a more readable display name for a version ID"""
    parts = version_id.split('_')
    if len(parts) >= 2:
        prefix = parts[0]
        version_num = parts[1].replace('v', '')
        type_name = prefix_to_name.get(prefix, prefix)
        try:
            # Try to convert version to integer for better display
            ver_num = int(version_num)
            return f"{type_name} (v{ver_num})"
        except ValueError:
            # If not a simple number, use as is
            return f"{type_name} ({version_num})"
    return version_id  # Fallback to original if pattern doesn't match

# Map prefixes to descriptive names
prefix_to_name = {
    'sl': 'Schema Linking',
    'diff': 'Difficulty Classification',
    'dir': 'Direct SQL Generation',
    'sbs': 'Step-by-Step SQL Generation',
    'sbscot': 'Step-by-Step with Chain-of-Thought',
    'sc': 'Self-Correction'
}

# Format prompt content for display
def format_prompt_content(content, format_type='markdown'):
    """Format prompt content for display"""
    if content is None or content == '':
        return "<em>Empty content</em>"
    
    if format_type == 'markdown':
        # Escape any HTML in the content and format as markdown code block
        return f"```\n{content}\n```"
    elif format_type == 'raw':
        # Return raw content with HTML escaping
        return f"<pre style='white-space: pre-wrap; word-wrap: break-word;'>{content}</pre>"
    else:
        return f"<pre style='white-space: pre-wrap; word-wrap: break-word;'>{content}</pre>"

# Export the current prompt version to HTML/Markdown
def export_current_prompt(prompt_type, prompt_version, format_type='markdown'):
    """Export the current prompt version to HTML/Markdown"""
    version_data = prompt_versions[prompt_version]
    
    if format_type == 'markdown':
        output = f"# {prefix_to_name.get(prompt_type, prompt_type)} - {prompt_version}\n\n"
        
        for field_name, field_content in version_data.items():
            if field_content is not None and field_content != '':
                output += f"## {field_name}\n\n```\n{field_content}\n```\n\n"
    else:  # HTML
        output = f"<h1>{prefix_to_name.get(prompt_type, prompt_type)} - {prompt_version}</h1>"
        
        for field_name, field_content in version_data.items():
            if field_content is not None and field_content != '':
                output += f"<h2>{field_name}</h2><pre style='white-space: pre-wrap; word-wrap: break-word;'>{field_content}</pre>"
    
    return output

## Prompt Viewer Widget

In [3]:
class PromptViewer:
    def __init__(self):
        self.grouped_versions = group_versions_by_prefix()
        self.prompt_types = list(self.grouped_versions.keys())
        
        # Create widgets
        self.prompt_type_dropdown = widgets.Dropdown(
            options=[(prefix_to_name.get(t, t), t) for t in self.prompt_types],
            description='Prompt Type:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='350px')
        )
        
        # Create version dropdown with descriptive names
        first_type_versions = self.grouped_versions[self.prompt_types[0]]
        version_options = [(get_version_display_name(v), v) for v in first_type_versions]
        
        self.version_dropdown = widgets.Dropdown(
            options=version_options,
            description='Version:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        # Add difficulty selector for step-by-step prompts
        self.difficulty_dropdown = widgets.Dropdown(
            options=[(level.capitalize(), level) for level in DifficultyLevel.get_valid_levels()],
            description='Difficulty:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='250px')
        )
        
        # Add step selector for step-by-step prompts
        self.step_dropdown = widgets.Dropdown(
            options=[
                ('Both Steps', 'both'),
                ('Planning Step', 'planning'),
                ('SQL Generation Step', 'sql_gen')
            ],
            description='Show Step:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='250px'),
            value='both'
        )
        
        # Add error type selector for self-correction prompts
        self.error_type_dropdown = widgets.Dropdown(
            options=[(err_type.replace('_', ' ').capitalize(), err_type) for err_type in SQLErrorType.get_valid_error_types()],
            description='Error Type:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='250px')
        )
        
        self.display_format = widgets.RadioButtons(
            options=['markdown', 'raw'],
            value='markdown',
            description='Display Format:',
            disabled=False,
            layout=widgets.Layout(width='300px')
        )
        
        self.export_button = widgets.Button(
            description='Export to Markdown',
            button_style='info',
            tooltip='Export current prompt to Markdown format',
            icon='download'
        )
        
        # Add button to show formatted prompt
        self.show_formatted_prompt_button = widgets.Button(
            description='Show Formatted Prompt',
            button_style='primary',
            tooltip='Show the complete formatted prompt',
            layout=widgets.Layout(width='200px')
        )
        
        # Add buttons for metadata and structure analysis
        self.show_metadata_button = widgets.Button(
            description='Show Metadata',
            button_style='info',
            tooltip='Show statistics about this prompt version',
            icon='info-circle',
            layout=widgets.Layout(width='150px')
        )
        
        self.analyze_structure_button = widgets.Button(
            description='Analyze Structure',
            button_style='info',
            tooltip='Analyze the sections in this prompt',
            icon='search',
            layout=widgets.Layout(width='150px')
        )
        
        self.field_selector = widgets.SelectMultiple(
            options=[],
            description='Fields:',
            disabled=False,
            layout=widgets.Layout(width='350px', height='300px')
        )
        
        self.content_output = widgets.Output(layout=widgets.Layout(border='1px solid #ddd', padding='10px'))
        self.export_output = widgets.Output()
        self.formatted_prompt_output = widgets.Output(layout=widgets.Layout(border='1px solid #ddd', padding='10px', max_height='500px', overflow='auto'))
        self.metadata_output = widgets.Output(layout=widgets.Layout(border='1px solid #ddd', padding='10px', max_height='800px', overflow='auto'))
        self.structure_output = widgets.Output(layout=widgets.Layout(border='1px solid #ddd', padding='10px', max_height='800px', overflow='auto'))
        
        # Set up event handlers
        self.prompt_type_dropdown.observe(self.on_prompt_type_change, names='value')
        self.version_dropdown.observe(self.on_version_change, names='value')
        self.field_selector.observe(self.on_field_select, names='value')
        self.display_format.observe(self.on_display_format_change, names='value')
        self.export_button.on_click(self.on_export_click)
        self.show_formatted_prompt_button.on_click(self.on_show_formatted_prompt_click)
        self.show_metadata_button.on_click(self.on_show_metadata_click)
        self.analyze_structure_button.on_click(self.on_analyze_structure_click)
        
        # Initial update
        self.update_version_dropdown()
        self.update_field_selector()
    
    def on_prompt_type_change(self, change):
        self.update_version_dropdown()
        self.update_additional_options_visibility()
    
    def update_version_dropdown(self):
        prompt_type = self.prompt_type_dropdown.value
        versions_list = self.grouped_versions[prompt_type]
        
        # Create options with display names
        version_options = [(get_version_display_name(v), v) for v in versions_list]
        
        self.version_dropdown.options = version_options
        self.update_field_selector()
        
    def update_additional_options_visibility(self):
        prompt_type = self.prompt_type_dropdown.value
        
        # Show/hide difficulty dropdown based on prompt type
        if prompt_type in ['sbs', 'sbscot']:
            self.difficulty_dropdown.layout.display = ''
            self.step_dropdown.layout.display = ''
        else:
            self.difficulty_dropdown.layout.display = 'none'
            self.step_dropdown.layout.display = 'none'
            
        # Show/hide error type dropdown based on prompt type
        if prompt_type == 'sc':
            self.error_type_dropdown.layout.display = ''
        else:
            self.error_type_dropdown.layout.display = 'none'
    
    def on_version_change(self, change):
        self.update_field_selector()
        # Clear the outputs when changing version
        with self.metadata_output:
            self.metadata_output.clear_output()
        with self.structure_output:
            self.structure_output.clear_output()
    
    def update_field_selector(self):
        version = self.version_dropdown.value
        if version in prompt_versions:
            fields = list(prompt_versions[version].keys())
            self.field_selector.options = fields
            
            # By default, select all fields
            self.field_selector.value = fields
            
            # Render the selected fields
            self.render_selected_fields()
    
    def on_field_select(self, change):
        self.render_selected_fields()
    
    def on_display_format_change(self, change):
        self.render_selected_fields()
        if change.new == 'markdown':
            self.export_button.description = 'Export to Markdown'
        else:
            self.export_button.description = 'Export to HTML'
    
    def on_export_click(self, b):
        prompt_type = self.prompt_type_dropdown.value
        version = self.version_dropdown.value
        format_type = self.display_format.value
        
        output = export_current_prompt(prompt_type, version, format_type)
        
        with self.export_output:
            self.export_output.clear_output()
            if format_type == 'markdown':
                display(Markdown(f"Export content (copy from here):\n\n```markdown\n{output}\n```"))
            else:
                display(HTML(f"<div>Export content (copy from here):</div><pre>{output}</pre>"))
    
    def on_show_formatted_prompt_click(self, b):
        version = self.version_dropdown.value
        
        with self.formatted_prompt_output:
            self.formatted_prompt_output.clear_output()
            
            try:
                formatted_prompt = self.get_formatted_prompt(version)
                if self.display_format.value == 'markdown':
                    display(Markdown(f"## Complete Formatted Prompt\n\n```\n{formatted_prompt}\n```"))
                else:
                    display(HTML(f"<h2>Complete Formatted Prompt</h2><pre style='white-space: pre-wrap; word-wrap: break-word;'>{formatted_prompt}</pre>"))
            except Exception as e:
                display(HTML(f"<div style='color:red'>Error formatting prompt: {str(e)}</div>"))
    
    def on_show_metadata_click(self, b):
        version = self.version_dropdown.value
        
        with self.metadata_output:
            self.metadata_output.clear_output()
            try:
                display(display_prompt_metadata(version))
            except Exception as e:
                display(HTML(f"<div style='color:red'>Error analyzing prompt metadata: {str(e)}</div>"))
    
    def on_analyze_structure_click(self, b):
        version = self.version_dropdown.value
        
        with self.structure_output:
            self.structure_output.clear_output()
            try:
                display(display_field_sections(version))
            except Exception as e:
                display(HTML(f"<div style='color:red'>Error analyzing prompt structure: {str(e)}</div>"))
    
    def get_formatted_prompt(self, version):
        """Format the prompt based on prompt type and version"""
        prompt_type = self.prompt_type_dropdown.value
        
        if prompt_type == 'sbs' or prompt_type == 'sbscot':
            # Get difficulty for step-by-step prompts
            difficulty = self.difficulty_dropdown.value
            step_option = self.step_dropdown.value
            
            # For step-by-step prompts, we would format differently based on difficulty
            version_data = prompt_versions[version]
            db_schema = version_data.get('db_schema', {})
            
            if difficulty == DifficultyLevel.SIMPLE:
                # Direct generation for simple queries
                prompt_format = version_data.get('sql_gen_prompt_format', '')
                task = version_data.get('sql_gen_task', '')
                context = version_data.get('sql_gen_context', '')
                final_instructions = version_data.get('sql_gen_final_instructions', '')
                
                db_description = get_db_schema_prompt(db_schema, ["tables_list"])
                return prompt_format.format(
                    general_task=task,
                    general_context=context,
                    db_schema=db_description,
                    final_instructions=final_instructions
                ) + "\n\n [User request]"
                
            elif difficulty == DifficultyLevel.MEDIUM:
                # Medium difficulty specific formatting
                if prompt_type == 'sbscot':
                    prompt_format = version_data.get('medium_sql_gen_prompt_format', '')
                    task = version_data.get('medium_sql_gen_task', '')
                    context = version_data.get('medium_sql_gen_context', '')
                    instructions = version_data.get('medium_sql_gen_instructions', '')
                    
                    db_description = get_db_schema_prompt(db_schema, ["tables_list"])
                    return prompt_format.format(
                        medium_generation_task=task,
                        medium_query_cntx=context,
                        db_schema=db_description,
                        medium_final_instructions=instructions
                    ) + "\n\n [User request]"
                else:
                    # For standard step-by-step
                    db_description = get_db_schema_prompt(db_schema, ["tables_list"])
                    
                    # First show the plan generation
                    plan_format = version_data.get('medium_plan_prompt_format', '')
                    plan_task = version_data.get('medium_plan_task', '')
                    plan_context = version_data.get('medium_plan_context', '')
                    plan_instructions = version_data.get('medium_plan_instructions', '')
                    
                    plan_prompt = plan_format.format(
                        medium_decomp_task=plan_task,
                        medium_query_cntx=plan_context,
                        db_schema=db_description,
                        medium_final_instructions=plan_instructions
                    ) + "\n\n [User request]"
                    
                    # Then show the SQL generation with plan
                    sql_format = version_data.get('medium_sql_gen_prompt_format', '')
                    sql_task = version_data.get('medium_sql_gen_task', '')
                    sql_context = version_data.get('medium_sql_gen_context', '')
                    sql_instructions = version_data.get('medium_sql_gen_instructions', '')
                    
                    sql_prompt = sql_format.format(
                        medium_generation_task=sql_task,
                        db_schema=db_description,
                        decomp_plan="[Generated Plan would be here]",
                        medium_final_instructions=sql_instructions
                    ) + "\n\n [User request]"
                    
                    # Return based on selected step option
                    if step_option == 'planning':
                        return f"# Plan Generation Prompt:\n\n{plan_prompt}"
                    elif step_option == 'sql_gen':
                        return f"# SQL Generation Prompt (with plan):\n\n{sql_prompt}"
                    else: # Both
                        return f"# Plan Generation Prompt:\n\n{plan_prompt}\n\n# SQL Generation Prompt (with plan):\n\n{sql_prompt}"
                
            elif difficulty == DifficultyLevel.ADVANCED:
                # Advanced difficulty specific formatting
                if prompt_type == 'sbscot':
                    prompt_format = version_data.get('adv_sql_gen_prompt_format', '')
                    task = version_data.get('adv_sql_gen_task', '')
                    context = version_data.get('adv_sql_gen_context', '')
                    instructions = version_data.get('adv_sql_gen_instructions', '')
                    
                    db_description = get_db_schema_prompt(db_schema, ["tables_list"])
                    return prompt_format.format(
                        adv_generation_task=task,
                        adv_query_cntx=context,
                        db_schema=db_description,
                        adv_final_instructions=instructions
                    ) + "\n\n [User request]"
                else:
                    # For standard step-by-step
                    db_description = get_db_schema_prompt(db_schema, ["tables_list"])
                    
                    # First show the plan generation
                    plan_format = version_data.get('adv_plan_prompt_format', '')
                    plan_task = version_data.get('adv_plan_task', '')
                    plan_context = version_data.get('adv_plan_context', '')
                    plan_instructions = version_data.get('adv_plan_instructions', '')
                    
                    plan_prompt = plan_format.format(
                        adv_decomp_task=plan_task,
                        adv_query_cntx=plan_context,
                        db_schema=db_description,
                        adv_final_instructions=plan_instructions
                    ) + "\n\n [User request]"
                    
                    # Then show the SQL generation with plan
                    sql_format = version_data.get('adv_sql_gen_prompt_format', '')
                    sql_task = version_data.get('adv_sql_gen_task', '')
                    sql_context = version_data.get('adv_sql_gen_context', '')
                    sql_instructions = version_data.get('adv_sql_gen_instructions', '')
                    
                    sql_prompt = sql_format.format(
                        adv_generation_task=sql_task,
                        db_schema=db_description,
                        decomp_plan="[Generated Plan would be here]",
                        adv_final_instructions=sql_instructions
                    ) + "\n\n [User request]"
                    
                    # Return based on selected step option
                    if step_option == 'planning':
                        return f"# Plan Generation Prompt:\n\n{plan_prompt}"
                    elif step_option == 'sql_gen':
                        return f"# SQL Generation Prompt (with plan):\n\n{sql_prompt}"
                    else: # Both
                        return f"# Plan Generation Prompt:\n\n{plan_prompt}\n\n# SQL Generation Prompt (with plan):\n\n{sql_prompt}"
        
        elif prompt_type == 'sc':
            # For self-correction prompts
            error_type = self.error_type_dropdown.value
            
            version_data = prompt_versions[version]
            db_schema = version_data.get('db_schema', {})
            
            db_description = get_db_schema_prompt(db_schema, ["tables_list"])
            
            if error_type == SQLErrorType.TIMEOUT:
                prompt_format = version_data.get('timeout_prompt_format', '')
            elif error_type == SQLErrorType.NOT_EXIST:
                prompt_format = version_data.get('not_exist_prompt_format', '')
            else:  # Default to schema error prompt for other errors
                prompt_format = version_data.get('schema_error_prompt_format', '')
            
            return prompt_format.format(
                Self_correction_task=version_data.get('general_task', ''),
                request="[User Query]",
                tab_schema=db_description,
                sql_query="[Generated SQL Query]",
                sql_error="[SQL Error Message]",
                final_instructions=version_data.get('final_instructions', ''),
                gen_cntx=version_data.get('general_context', '')
            )
        
        else:
            # For other prompt types, use the load_prompt function
            return load_prompt(version)
    
    def render_selected_fields(self):
        version = self.version_dropdown.value
        selected_fields = self.field_selector.value
        format_type = self.display_format.value
        
        with self.content_output:
            self.content_output.clear_output()
            
            if version in prompt_versions:
                version_data = prompt_versions[version]
                
                for field in selected_fields:
                    if field in version_data:
                        field_content = version_data[field]
                        
                        # Special handling for db_schema field
                        if field == 'db_schema' and isinstance(field_content, dict):
                            # Show both raw and formatted versions of db_schema
                            if format_type == 'markdown':
                                # Also show the formatted version
                                try:
                                    formatted_schema = get_db_schema_prompt(field_content, list(field_content.keys()))
                                    display(Markdown(f"### {field} (Formatted)\n```\n{formatted_schema}\n```"))
                                except Exception as e:
                                    display(Markdown(f"**Error formatting schema:** {str(e)}"))
                            else:
                                # Also show the formatted version
                                try:
                                    formatted_schema = get_db_schema_prompt(field_content, list(field_content.keys()))
                                    display(HTML(f"<h3>{field} (Formatted)</h3><pre style='white-space: pre-wrap; word-wrap: break-word;'>{formatted_schema}</pre>"))
                                except Exception as e:
                                    display(HTML(f"<div style='color:red'>Error formatting schema: {str(e)}</div>"))
                        else:
                            # Regular field rendering
                            formatted_content = format_prompt_content(field_content, format_type)
                            
                            if format_type == 'markdown':
                                display(Markdown(f"### {field}\n{formatted_content}"))
                            else:
                                display(HTML(f"<h3>{field}</h3>{formatted_content}"))
    
    def display(self):
        # Layout the widgets
        primary_controls = widgets.VBox([
            self.prompt_type_dropdown,
            self.version_dropdown,
            self.display_format,
            self.difficulty_dropdown,
            self.step_dropdown,
            self.error_type_dropdown,
            widgets.HBox([self.export_button, self.show_formatted_prompt_button]),
            widgets.HBox([self.show_metadata_button, self.analyze_structure_button])
        ])
        
        # Hide additional options initially
        self.difficulty_dropdown.layout.display = 'none'
        self.step_dropdown.layout.display = 'none'
        self.error_type_dropdown.layout.display = 'none'
        
        controls_box = widgets.HBox([
            primary_controls,
            widgets.VBox([
                widgets.Label('Select fields to display:'),
                self.field_selector
            ])
        ])
        
        # Create tabs for different views
        tab_fields = widgets.Output()
        tab_formatted = widgets.Output()
        tab_analysis = widgets.Output()
        
        with tab_fields:
            display(widgets.VBox([
                widgets.Label('Field Content:'),
                self.content_output
            ]))
        
        with tab_formatted:
            display(widgets.VBox([
                widgets.Label('Formatted Prompt:'),
                self.formatted_prompt_output
            ]))
        
        with tab_analysis:
            display(widgets.VBox([
                widgets.HTML("<h3>Advanced Analysis</h3>"),
                widgets.HTML("<p>View metadata and structure of the current prompt version:</p>"),
                widgets.HBox([
                    widgets.VBox([
                        widgets.HTML("<h4>Prompt Metadata</h4>"),
                        self.metadata_output
                    ]),
                    widgets.VBox([
                        widgets.HTML("<h4>Structure Analysis</h4>"),
                        self.structure_output
                    ])
                ])
            ]))
        
        tabs = widgets.Tab(children=[tab_fields, tab_formatted, tab_analysis])
        tabs.set_title(0, 'Individual Fields')
        tabs.set_title(1, 'Formatted Prompt')
        tabs.set_title(2, 'Analysis')
        
        # Main layout
        main_layout = widgets.VBox([
            controls_box,
            tabs,
            self.export_output
        ])
        
        # Update visibility of additional dropdowns
        self.update_additional_options_visibility()
        
        display(main_layout)

## Create and Display the Prompt Viewer

In [4]:
# Create and display the main prompt viewer
viewer = PromptViewer()
viewer.display()

## Summary of Prompt Types

Here's a summary of the different prompt types used in the ALeRCE Text-to-SQL pipeline:

In [5]:
def display_prompt_summary():
    # Create a summary of prompt types and their versions
    grouped = group_versions_by_prefix()
    
    data = []
    for prefix, versions in grouped.items():
        data.append({
            'Type': prefix_to_name.get(prefix, prefix),
            'Prefix': prefix,
            'Versions': ', '.join(versions),
            'Count': len(versions)
        })
    
    df = pd.DataFrame(data)
    return df

display_prompt_summary()

,Type,Prefix,Versions,Count
0,Schema Linking,sl,"sl_v0, sl_v1, sl_v2, sl_v3",4
1,Difficulty Classification,diff,"diff_v0, diff_v7, diff_v8",3
2,Direct SQL Generation,dir,"dir_v0, dir_v10, dir_v8, dir_v9",4
3,Step-by-Step SQL Generation,sbs,"sbs_v0, sbs_v4",2
4,Step-by-Step with Chain-of-Thought,sbscot,"sbscot_v0, sbscot_v1",2
5,Self-Correction,sc,"sc_v0, sc_v3",2


## Advanced Analysis (Optional)

You can use this section to do more advanced analysis of your prompt versions.

In [6]:
def compare_versions(version1, version2):
    """Compare two prompt versions and show their differences"""
    if version1 not in prompt_versions or version2 not in prompt_versions:
        return "One or both versions not found"
    
    v1 = prompt_versions[version1]
    v2 = prompt_versions[version2]
    
    # Get all unique keys
    all_keys = set(list(v1.keys()) + list(v2.keys()))
    
    comparison = {}
    for key in all_keys:
        if key in v1 and key in v2:
            if v1[key] == v2[key]:
                comparison[key] = "Same in both versions"
            else:
                comparison[key] = "Different between versions"
        elif key in v1:
            comparison[key] = f"Only in {version1}"
        else:
            comparison[key] = f"Only in {version2}"
    
    return pd.DataFrame({'Field': list(comparison.keys()), 'Status': list(comparison.values())})

def compare_versions_side_by_side(version1, version2, field=None, show_identical=False):
    """
    Compare two prompt versions side by side with highlighted differences
    
    Args:
        version1: First version to compare
        version2: Second version to compare
        field: Specific field to compare. If None, all fields will be compared
        show_identical: Whether to show fields that are identical in both versions
    
    Returns:
        HTML output with highlighted differences
    """
    import difflib
    import json
    
    if version1 not in prompt_versions or version2 not in prompt_versions:
        return HTML("<div style='color:red'>One or both versions not found</div>")
    
    v1 = prompt_versions[version1]
    v2 = prompt_versions[version2]
    
    # Get all unique keys
    all_keys = set(list(v1.keys()) + list(v2.keys()))
    
    if field is not None:
        if field not in v1 or field not in v2:
            return HTML(f"<div style='color:red'>Field '{field}' not found in one or both versions</div>")
        all_keys = [field]
    
    # Count statistics
    different_fields = []
    only_in_v1 = []
    only_in_v2 = []
    identical_fields = []
    
    for key in sorted(all_keys):
        if key in v1 and key in v2:
            if v1[key] == v2[key]:
                identical_fields.append(key)
            else:
                different_fields.append(key)
        elif key in v1:
            only_in_v1.append(key)
        else:
            only_in_v2.append(key)
    
    # Get display names
    display_name1 = get_version_display_name(version1)
    display_name2 = get_version_display_name(version2)
    
    # Initialize an object to collect exportable diff data
    diff_data = {
        "version1": {"id": version1, "display": display_name1},
        "version2": {"id": version2, "display": display_name2},
        "fields": {}
    }
    
    # Create a unique ID for this comparison
    comparison_id = f"comp_{version1}_{version2}_{field if field else 'all'}"
    
    # Add copy-to-clipboard functionality
    clipboard_js = f"""
    <script>
    function copyDiffToClipboard_{comparison_id}() {{
        const diffData = {{"version1": "{version1}", "version2": "{version2}", "fields": {{}}}};
        
        // Add the field comparisons
        
        // Convert to string and copy to clipboard
        const diffString = JSON.stringify(diffData, null, 2);
        navigator.clipboard.writeText(diffString)
            .then(() => {{
                document.getElementById("copy_status_{comparison_id}").textContent = "Copied to clipboard!";
                setTimeout(() => {{
                    document.getElementById("copy_status_{comparison_id}").textContent = "";
                }}, 2000);
            }})
            .catch(err => {{
                document.getElementById("copy_status_{comparison_id}").textContent = "Error copying to clipboard";
                console.error('Failed to copy: ', err);
            }});
    }}
    </script>
    """
    
    output_html = f"""
    <div style='font-family: Arial, sans-serif; margin-bottom: 20px;'>
        <h2 style='color: #333; text-align: center;'>Comparing Prompts</h2>
        <div style='display: flex; justify-content: space-between; margin-bottom: 15px;'>
            <div style='flex: 1; text-align: center; background-color: #e6f0ff; padding: 10px; border-radius: 5px; margin-right: 5px;'>
                <span style='font-weight: bold; color: #0066cc; font-size: 1.2em;'>{display_name1}</span>
                <div style='font-size: 0.8em; color: #666;'>({version1})</div>
            </div>
            <div style='flex: 1; text-align: center; background-color: #e6ffe6; padding: 10px; border-radius: 5px; margin-left: 5px;'>
                <span style='font-weight: bold; color: #009933; font-size: 1.2em;'>{display_name2}</span>
                <div style='font-size: 0.8em; color: #666;'>({version2})</div>
            </div>
        </div>
        
        <div style='background-color: #f8f8f8; padding: 15px; border-radius: 8px; margin-bottom: 20px;'>
            <h3 style='margin-top: 0;'>Summary:</h3>
            <div style='display: flex; flex-wrap: wrap;'>
                <div style='flex: 1; min-width: 200px;'>
                    <div style='font-weight: bold; color: #990000;'>Different fields: {len(different_fields)}</div>
                    <div style='font-size: 0.9em; color: #666; margin-top: 5px;'>{', '.join(different_fields) if different_fields else 'None'}</div>
                </div>
                <div style='flex: 1; min-width: 200px;'>
                    <div style='font-weight: bold; color: #0066cc;'>Only in {display_name1}: {len(only_in_v1)}</div>
                    <div style='font-size: 0.9em; color: #666; margin-top: 5px;'>{', '.join(only_in_v1) if only_in_v1 else 'None'}</div>
                </div>
                <div style='flex: 1; min-width: 200px;'>
                    <div style='font-weight: bold; color: #009933;'>Only in {display_name2}: {len(only_in_v2)}</div>
                    <div style='font-size: 0.9em; color: #666; margin-top: 5px;'>{', '.join(only_in_v2) if only_in_v2 else 'None'}</div>
                </div>
                <div style='flex: 1; min-width: 200px;'>
                    <div style='font-weight: bold; color: #666;'>Identical fields: {len(identical_fields)}</div>
                    <div style='font-size: 0.9em; color: #666; margin-top: 5px;'>{(identical_fields[0] + "..." if len(identical_fields) > 1 else ", ".join(identical_fields)) if identical_fields else 'None'} {'(hidden)' if not show_identical else ''}</div>
                </div>
            </div>
            
            <div style='margin-top: 15px; display: flex; justify-content: space-between; align-items: center;'>
                <div>
                    <button onclick="copyDiffToClipboard_{comparison_id}()" 
                            style='background-color: #0066cc; color: white; border: none; padding: 5px 10px; border-radius: 4px; cursor: pointer;'>
                        Copy Diff Data
                    </button>
                    <span id="copy_status_{comparison_id}" style='margin-left: 10px; color: #009933;'></span>
                </div>
                <div style='color: #666; font-size: 0.9em;'>
                    Showing {field if field else 'all fields'} • {show_identical and 'Including' or 'Excluding'} identical fields
                </div>
            </div>
        </div>
    </div>
    """
    
    # Add CSS styles for diff highlighting
    css_styles = """
    <style>
    .diff-table {width:100%; border-collapse:collapse; font-family:monospace; font-size:14px; border:1px solid #ddd; margin-bottom:25px; box-shadow: 0 2px 5px rgba(0,0,0,0.1);}
    .diff-header {padding:12px; text-align:center; font-weight:bold; border-bottom:2px solid #ccc; font-size:16px;}
    .diff-header-v1 {background-color:#e6f0ff; color:#0066cc;}
    .diff-header-v2 {background-color:#e6ffe6; color:#009933;}
    .diff-cell {vertical-align:top; padding:0; background-color:#f8f8f8;}
    .diff-cell-left {border-right:2px solid #ccc;}
    .diff-line {padding:3px 5px; white-space:pre-wrap; overflow-x:auto; line-height:1.5;}
    .diff-removed {background-color:#ffcccc !important; color:#990000; text-decoration:line-through; text-decoration-color:#990000; text-decoration-thickness:1px; position:relative;}
    .diff-removed::after {content:''; position:absolute; left:0; top:0; height:100%; width:3px; background-color:#990000;}
    .diff-added {background-color:#ccffcc !important; color:#006600; border-left:3px solid #006600; font-weight:500;}
    .diff-empty {color:#aaaaaa; font-style:italic;}
    .field-header {background-color:#f0f0f0; padding:10px; margin-bottom:5px; border-radius:5px; font-weight:bold;}
    </style>
    """
    output_html += css_styles
    
    # Process all keys for comparison
    displayed_fields = []
    
    for key in sorted(all_keys):
        # Determine status
        if key in v1 and key in v2:
            if v1[key] == v2[key]:
                status = "identical"
                if not show_identical and field is None:  # Skip identical fields when comparing all
                    continue
            else:
                status = "different"
        elif key in v1:
            status = "only_in_v1"
        else:
            status = "only_in_v2"
        
        displayed_fields.append(key)
        
        # Create field header
        output_html += f"<div class='field-header'>Field: {key}</div>"
        
        # Generate side-by-side diff for different content
        if key in v1 and key in v2:
            # Convert content to string if it's not already
            content1 = str(v1[key]) if v1[key] is not None else ""
            content2 = str(v2[key]) if v2[key] is not None else ""
            
            if status == "different":
                # Save content for export
                diff_data["fields"][key] = {"content1": content1, "content2": content2}
                
                # Process the content for highlighting differences
                def highlight_diff(text1, text2):
                    """Highlight differences between two texts"""
                    lines1 = text1.splitlines()
                    lines2 = text2.splitlines()
                    
                    # Use difflib's HtmlDiff for side-by-side comparison
                    # This provides more accurate and visually appealing differences
                    differ = difflib.HtmlDiff(tabsize=4, wrapcolumn=80)
                    
                    # Generate the diff HTML
                    html_diff = differ.make_file(lines1, lines2, fromdesc=display_name1, todesc=display_name2, context=True)
                    
                    # Remove the default CSS and keep only the table
                    start_table = html_diff.find("<table")
                    end_table = html_diff.find("</table>", start_table) + 8
                    diff_table = html_diff[start_table:end_table]
                    
                    # Replace default styling with our own classes
                    diff_table = diff_table.replace('class="diff"', 'class="diff-table"')
                    diff_table = diff_table.replace('<th', '<th class="diff-header"')
                    diff_table = diff_table.replace('class="diff_add"', 'class="diff-added"')
                    diff_table = diff_table.replace('class="diff_chg"', 'class="diff-changed"')
                    diff_table = diff_table.replace('class="diff_sub"', 'class="diff-removed"')
                    
                    return diff_table
                
                # Get custom HTML diff with better highlighting
                word_diff_html = highlight_diff(content1, content2)
                
                # Add the highlighted diff to the output
                output_html += f"""
                <div style='margin-bottom: 20px;'>
                    {word_diff_html}
                </div>
                """
            else:
                # For identical content, just show it once
                diff_data["fields"][key] = {"content": content1}
                
                output_html += f"""
                <div style='padding: 15px; background-color: #f8f8f8;'>
                    <pre style='margin: 0; white-space: pre-wrap; overflow-x: auto; font-size: 14px;'>{html.escape(content1)}</pre>
                </div>
                """
        else:
            # Show content that exists in only one version
            content = None
            version = None
            bg_color = None
            
            if key in v1:
                content = v1[key]
                version = version1
                display_name = display_name1
                bg_color = "#e6f0ff"  # Light blue for version1
                diff_data["fields"][key] = {"content1": str(content) if content is not None else ""}
            elif key in v2:
                content = v2[key]
                version = version2
                display_name = display_name2
                bg_color = "#e6ffe6"  # Light green for version2
                diff_data["fields"][key] = {"content2": str(content) if content is not None else ""}
                
            if content is not None:
                output_html += f"""
                <div style='padding: 10px; background-color: {bg_color}; margin-bottom: 20px;'>
                    <div style='margin-bottom: 5px;'><strong>Content in {display_name} ({version}):</strong></div>
                    <pre style='margin: 0; white-space: pre-wrap; overflow-x: auto; background-color: #f8f8f8; padding: 10px; border-radius: 3px;'>{html.escape(str(content))}</pre>
                </div>
                """
    
    if not displayed_fields:
        output_html += "<p>No fields to display based on current selection.</p>"
    
    # Add JavaScript to handle copying diff data
    js_template = """
    <script>
    document.addEventListener("DOMContentLoaded", function() {
        const diffData = {"version1": "%s", "version2": "%s", "fields": {}};
        
        // Add the field comparisons
    });
    </script>
    """ % (version1, version2)
    
    # Create the JavaScript field data assignments
    field_assignments = []
    for field, data in diff_data["fields"].items():
        field_assignments.append(f'diffData.fields["{field}"] = {json.dumps(data)};')
    
    js_with_data = js_template.replace("// Add the field comparisons", "\n        ".join(field_assignments))
    
    # Add the JavaScript to the output
    output_html += js_with_data
    
    return HTML(output_html)

## Prompt Version Comparison

Use the comparison widget below to compare different prompt versions side by side with highlighted differences.

In [8]:
compare_versions_side_by_side('dir_v0', 'dir_v8')

In [ ]:
# Create and display the comparison widget
# comparison_widget = create_comparison_widget()
# display(comparison_widget)

NameError: name 'create_comparison_widget' is not defined

## Features Guide

### Main Prompt Viewer
The prompt viewer provides several ways to explore and analyze prompts:

1. **Individual Fields Tab**: View the raw content of each field in the prompt
2. **Formatted Prompt Tab**: See how the prompt would look when sent to the LLM
3. **Analysis Tab**: Get insights about the prompt structure and content

### Version Comparison
The comparison tool makes it easy to see differences between prompt versions:

- Red highlights show content removed in the newer version
- Green highlights show content added in the newer version
- Summary statistics show which fields differ between versions

### Analysis Features
- **Show Metadata**: View statistics about the prompt (field counts, word counts, etc.)
- **Analyze Structure**: See how the prompt is organized into sections
- **Quick Example**: Try a pre-configured comparison to see how the tools work

These tools make it easier to understand, analyze, and improve your prompts.